In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 📥 Importing needed libraries

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# 📂 Loading dataset

In [ ]:
df = pd.read_csv('../input/hard-drive-test-data/harddrive.csv')
print(df.shape)
df.head()

In [ ]:
df = df.loc[:, ~df.isnull().all()]
print(df.shape)

In [ ]:
# number of hdd
print("number of hdd:", df['serial_number'].value_counts().shape) 

# number of different types of harddrives
print("number of different harddrives", df['model'].value_counts().shape)

In [ ]:
failed_hdds = df.loc[df.failure==1]["serial_number"]
len(failed_hdds)

In [ ]:
df = df.loc[df["serial_number"].isin(failed_hdds)]
df.shape

# ⏰ Finding the RUL (Remaining Useful Life)

In [ ]:
df["end_date"] = df.groupby("serial_number")["date"].transform("max")
print(df["end_date"])

In [ ]:
df["end_date"] = pd.to_datetime(df["end_date"])
df["date"] = pd.to_datetime(df["date"])
print(df["end_date"])
print(df["date"])

In [ ]:
df["date_diff"] = df["end_date"] - df["date"]
df["date_diff"].describe()
print(df["date_diff"])

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# df[df["serial_number"]=='Z300ZST1'].shape
# # print(df["serial_number"]=='Z300ZST1')
# # print(df[].shape)
# print(df)

# 🗑️ Getting rid of NaN values (Cleaning)


In [ ]:
df_notna = df[df.columns[~(df.isna().sum().values/len(df) > 0.05)]]
print(df_notna)

In [ ]:
df_notna.dropna(inplace=True)
print(df_notna.shape)

In [ ]:
df = df_notna.copy()
print(df.shape)

In [ ]:
df.head()
df.reset_index(inplace=True, drop=True)# no insertion in DataFrame
print(df.head())# sorting

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

# 🏗️ Dropping unwanted columns, Splitting Data and Training Model

In [ ]:
df = df.drop(['date', 'serial_number', 'model','end_date'], axis=1)
df.head()

In [ ]:
Y = df["date_diff"].dt.days # with date diff
X = df.drop(["date_diff"],axis=1)
print(X)# without date diff
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)
# print(X_train)
# print(X_test)
# print(y_train)
# print(y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RF_model=RandomForestRegressor(n_estimators=100,random_state=1)
RF_model.fit(X_train, y_train)# fitting model

In [ ]:
y_pred = RF_model.predict(X_test)
RF_model.score(X_test, y_test)# Testing